<a href="https://colab.research.google.com/github/FranciscoFoz/Projeto_Final_Bootcamp_Data_Science_Alura/blob/main/Notebooks/Notebook_de_Funcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color= MediumSpringGreen		size = 6> Bootcamp Data Science Alura - Projeto Final </font>

# <font color= MediumSpringGreen		size = 6> Notebook de funções

4º Notebook

## <font color= MediumSpringGreen> 1. Funções de manipulação de dados


### <font color= MediumSpringGreen> 1.1 remover_pacientes_com_primeira_janela_positiva_para_UTI

Esta função irá remover os pacientes que possuam resultado positivo(1) para entrada na UTI dentro da primeira janela (0-2)

In [ ]:
def remover_pacientes_com_primeira_janela_positiva_para_UTI(dados):
  a_remover = dados.query("WINDOW == '0-2' and ICU == 1")['PATIENT_VISIT_IDENTIFIER'].values            #Query dos pacientes que possuam a primeira janela e resultado positivo
  dados_limpos = dados.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")                              #Query removendo os pacientes na condição acima
  return dados_limpos

### <font color= MediumSpringGreen> 1.2 preenche_tabela
Esta função irá preencher a tabela onde esteja NAN com os valores anteriores e depois com os posteriores.

In [ ]:
def preenche_tabela(dados):
  features_continuas_colunas = dados.iloc[:,4:-2].columns                                                                                                   #Selecionar colunas de variáveis contínuas
  features_continuas = dados.groupby('PATIENT_VISIT_IDENTIFIER', as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')  #Preencher os dados NAN da tabela
  features_categoricas = dados.iloc[:,:4]                                                                                                                   #Selecionar as categóricas
  saida = dados.iloc[:, -2:]                                                                                                                                #Selecionar os dados de saída
  dados_finais = pd.concat([features_categoricas,features_continuas,saida],ignore_index=True,axis=1)                                                        #Concatenar os dados anteriores
  dados_finais.columns = dados.columns                                                                                                                      #Renomear as colunas
  return dados_finais

### <font color= MediumSpringGreen> 1.2 prepare_window
Esta função irá localizar todas as linhas que possuam janela entre "0-2" e UTI igual a 1, retornando as linhas.

In [ ]:
def prepare_window(rows):                                                   
  if(np.any(rows['ICU'])):                                                      
    rows.loc[rows['WINDOW']=='0-2','ICU'] = 1                                   #Condição se para localizar as linhas com os filtros desejados.
  return rows.loc[rows['WINDOW'] == '0-2']

### <font color= MediumSpringGreen> 1.3 excluir_coluna_id_paciente
Esta função irá excluir a coluna do ID do paciente.

In [ ]:
def excluir_coluna_id_paciente(dados):
  dados_sem_coluna_id_paciente = dados.drop(['PATIENT_VISIT_IDENTIFIER'],axis=1)      #Excluir coluna do id do paciente
  return dados_sem_coluna_id_paciente                               

### <font color= MediumSpringGreen> 1.4 transformar_AGE_PERCENTIL_em_dados_categoricos
Esta função irá excluir a coluna do ID do paciente.

In [ ]:
def transformar_AGE_PERCENTIL_em_dados_categoricos(dados):
  dados.AGE_PERCENTIL = dados.AGE_PERCENTIL.astype('category').cat.codes                              #Transformar coluna AGE PECENTIL em dados categóricos
  return dados

### <font color= MediumSpringGreen> 1.5 remover_variaveis_correlacionadas
Esta função irá filtrar os dados entre as variáveis, excluindo: "AGE_ABOVE65","AGE_PERCENTIL", "GENDER", "WINDOW" e "ICU".

Criará uma matriz de correlação entre eles, permanecendo apenas com o triângulo superior.

Excluirá as colunas com correção passada como parâmetro.


In [ ]:
def remover_variaveis_correlacionadas(dados,valor_de_corte):

  matriz_corr = dados.iloc[:,3:-2].corr().abs()                                                               #Filtrar dados correlacionando-os e transformando-os em números absolutos.
  matriz_superior = matriz_corr.where(np.triu(np.ones(matriz_corr.shape),k=1).astype(np.bool))                #Criar matriz superior da matriz de correlação
  excluir = [coluna for coluna in  matriz_superior if any(matriz_superior[coluna] > valor_de_corte)]          #Selecionar colunas dentro da matriz superior que contenham alta correlação

  return dados.drop(excluir,axis=1)                                                                           #Retornar o dataframe sem as colunas com alta correlação

### <font color= MediumSpringGreen> 1.6 limpar_colunas_com_valores_unicos
Esta função irá excluir as colunas que possuíam valores únicos de -1.

In [1]:
def limpar_colunas_com_valores_unicos(dados):
  dados = dados.drop(['ALBUMIN_DIFF','BE_ARTERIAL_DIFF','BE_VENOUS_DIFF','BIC_ARTERIAL_DIFF','BIC_VENOUS_DIFF','BILLIRUBIN_DIFF','BLAST_DIFF','CALCIUM_DIFF',
                            'CREATININ_DIFF','FFA_DIFF','GGT_DIFF','GLUCOSE_DIFF','HEMATOCRITE_DIFF', 'HEMOGLOBIN_DIFF','INR_DIFF','LACTATE_DIFF','LEUKOCYTES_DIFF',
                            'LINFOCITOS_DIFF', 'NEUTROPHILES_DIFF','P02_ARTERIAL_DIFF','P02_VENOUS_DIFF','PC02_ARTERIAL_DIFF','PC02_VENOUS_DIFF','PCR_DIFF','PH_ARTERIAL_DIFF',
                            'PH_VENOUS_DIFF','PLATELETS_DIFF','POTASSIUM_DIFF','SAT02_ARTERIAL_DIFF','SAT02_VENOUS_DIFF','SODIUM_DIFF','TGO_DIFF','TGP_DIFF','TTPA_DIFF',
                            'UREA_DIFF','DIMER_DIFF','BLOODPRESSURE_DIASTOLIC_DIFF', 'BLOODPRESSURE_SISTOLIC_DIFF','HEART_RATE_DIFF', 'RESPIRATORY_RATE_DIFF', 'TEMPERATURE_DIFF','OXYGEN_SATURATION_DIFF',
                            'RESPIRATORY_RATE_MIN','BLOODPRESSURE_DIASTOLIC_MAX'],
                           axis=1)
  return dados

## <font color= MediumSpringGreen> 2. Funções de criação de visualizações


### <font color= MediumSpringGreen> 2.1 grafico_total_pacientes
Esta função criará um gráfico de cartão com o total de pacientes

In [ ]:
def grafico_total_pacientes():

  plt.figure(figsize=(7, 3),                                                    #Ajustando o tamanho
            facecolor='white')                                                  #Escolhendo a cor do fundo

  for spine in plt.gca().spines.values():                                       #Removendo os eixos do gráfico
      spine.set_visible(False)

  plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False) #Excluindo os valores dos eixos


  plt.text(0, 1.03,                                                             #Ajustando posição dos eixos
          'Número total de pacientes',                                          #Colocando o título
          fontsize=25,                                                          #Formatando o tamanho  
          color='black',                                                        #Formatando a cor
          weight="bold")                                                        #Formatando em negrito

  plt.text(0, 0.25,                                                             #Ajustando posição dos eixos
          '$\\bf{353}$',                                                        #Colocando número com formatação em negrito com auxílio de regex                     
          fontsize=130,                                                         #Formatando o tamanho
          color='#4682B4')                                                      #Formatando a cor
  plt.show()

### <font color= MediumSpringGreen> 2.2 grafico_quantidade_entrada_e_nao_entrada_UTI
Esta função criará um gráfico de com a quantidade de pacientes que deram e não deram entrada na UTI.

In [ ]:
def grafico_quantidade_entrada_e_nao_entrada_UTI():
                                           
  tabela_freq_entrada_UTI = pd.DataFrame(dados_sem_valores_unicos['ICU'].value_counts())     #Criar DataFrame com a quantidade
  tabela_freq_entrada_UTI.columns = ['UTI']                                               #Renomear coluna
  tabela_freq_entrada_UTI.index = ['Não','Sim']                                           #Renomear índice

  tabela_freq_entrada_UTI.T.plot(kind='barh',                                    #Plotar o gráfico
                                      stacked=True,                              #Empilhar as barras
                                      figsize=(15,6),                            #Ajustar o tamanho
                                      color=['#808080','#800000'],               #Escolher a cor
                                      fontsize=15,                               #Ajustar o tamanho da fonte
                                      legend=None)                               #Excluir legenda

  for spine in plt.gca().spines.values():                                        #Remover os eixos do gráfico
      spine.set_visible(False)

  quantidade_sem_entrada = tabela_freq_entrada_UTI['UTI'][0]                                 
  quantidade_com_entrada = tabela_freq_entrada_UTI['UTI'][1]

  sem_entrada_percentual = str(round(quantidade_sem_entrada/tabela_freq_entrada_UTI['UTI'].sum(),2)*100)   
  com_entrada_percentual = str(round(quantidade_com_entrada/tabela_freq_entrada_UTI['UTI'].sum(),2)*100)    


                        
  plt.text(quantidade_sem_entrada/2.2,0,                                         #Escolher a posição x e y do texto de %
          sem_entrada_percentual + '%',                                          #Plotar o texto com o %
          fontweight ='bold',                                                    #Formatar para negrito
          fontsize = 20)                                                         #Aumentar a fonte

  plt.text(tabela_freq_entrada_UTI['UTI'].sum() - quantidade_sem_entrada/1.8,0,  #Escolher a posição x e y do texto
          com_entrada_percentual + '%',                                          #Plotar o texto com o %
          fontweight ='bold',                                                    #Formatar para negrito
          fontsize = 20)                                                         #Aumentar a fonte

  plt.yticks([])                                                                 #Excluir valores do eixo y
  plt.xticks([])                                                                 #Excluir valores do eixo x

  plt.text(-0.3, -0.4,                                                           #Escolher posição   
          'Do total de $\\bf{ 353 \\ pacientes}$.' ,                             #Texto com regex
          fontsize=25)                                                           #Escolher tamanho do texto                                            

  plt.text(-0.3, -0.5,                                                           #Escolher posição  
          '$\\bf{ 190 \\ não \\ tiveram \\ entrada}$.' ,                         #Texto com regex
          fontsize=25,                                                           #Escolher tamanho do texto 
          color='#808080')                                                       #Escolher cor

  plt.text(-0.3, -0.6,                                                           #Escolher posição
          'e $\\bf{ 163 \\ tiveram \\ entrada}$.' ,                              #Texto com regex
          fontsize=25,                                                           #Escolher tamanho do texto
          color='#800000')                                                       #Escolher cor
          
  plt.title('Quantidade de pacientes com entrada na UTI',  #Colocar o título
            fontsize=30,                                                         #Tamanho do texto
            loc='left',                                                          #Ajustar do lado esquerdo
            color='black')                                                       #Escolher a cor

  plt.show()

### <font color= MediumSpringGreen> 2.3 grafico_abaixo_acima_65anos
Esta função criará o gráfico comparativo entre os pacientes abaixo e acima de 65 anos de idade que tiveram ou não entrada na UTI.

In [ ]:
def grafico_abaixo_acima_65anos():

  tabela_freq = dados_sem_valores_unicos[['AGE_ABOVE65','ICU']].value_counts().reset_index()  #Criar DataFrame com a quantidade
  tabela_freq['AGE_ABOVE65'] = tabela_freq['AGE_ABOVE65'].map({0:'Abaixo',1:'Acima'})      #Mapear para substituir os valores de 0 e 1
  tabela_freq.columns = ['ACIMA_65_ANOS','UTI','QTD']                                      #Modificar as colunas
  tabela_freq = pd.pivot_table(tabela_freq,'QTD','ACIMA_65_ANOS','UTI')                    #Pivotar a tabela


  g = tabela_freq.plot(kind='bar',                                                         #Criar gráfico
                      color=['#808080']+['#800000'],                                       #Escolher cores das barras
                      figsize=(15, 8),                                                     #Escolher tamanho da figura
                      ylabel=False,                                                        #Excluir rótulo do eixo y y
                      legend=None)                                                         #Excluir legenda

  for rotulo in g.containers:                                                              #Criar rótulo de dados nas barras
      g.bar_label(rotulo,fontsize=15,padding=5)

  for spine in plt.gca().spines.values():                                                  #Remover os eixos do gráfico
      spine.set_visible(False)
                        
  plt.xticks(rotation=0, fontsize=15)                                                      #Excluir valores do eixo x
  plt.yticks([])                                                                           #Aumentar a fonte do eixo y
  plt.xlabel(None)                                                                         #Excluir rótulo do eixo y
  plt.ylabel(None)                                                                         #Excluir rótulo do eixo y



  plt.title('Comparativo entre pacientes abaixo e acima de 65 anos'+ 2*('\n'),fontsize=25,loc='left')           #Colocar o título
  plt.text(-0.5,145,'A entrada na UTI é maior em pacientes com idade superior',fontsize=20,color='#800000')     #Colocar o subtítulo

  plt.show()

### <font color= MediumSpringGreen> 2.4 faixa_etaria_entrada_UTI
Esta função criará o gráfico comparativo entre as faixas etárias entre os pacientes que tiveram ou não entrada na UTI.

In [ ]:
def faixa_etaria_entrada_UTI():
  rotulo = {5:'51-60 anos', 8:'81-90 anos', 0:'0-10 anos', 3:'31-40 anos', 6:'61-70 anos', 1:'11-20 anos', 4:'41-50 anos', 7:'71-80 anos',2:'21-30 anos', 9:'Acima de 90 anos'}  #Rótulos da faixa etária
  tabela_freq = pd.DataFrame(data={'FAIXA_ETARIA': list(dados_sem_valores_unicos['AGE_PERCENTIL']), 'UTI': list(dados_sem_valores_unicos['ICU'])})                                     #Criar dataframe
  tabela_freq['FAIXA_ETARIA'] = tabela_freq['FAIXA_ETARIA'].map(rotulo)                                                                                                          #Mapear valores dos rótulos
  tabela_freq = tabela_freq.value_counts().reset_index()                                                                                                                         #Criar frequência dos valores
  tabela_freq.columns = ['FAIXA_ETARIA','UTI','QTD']                                                                                                                             #Renomear colunas
  tabela_freq = pd.pivot_table(tabela_freq,'QTD','FAIXA_ETARIA','UTI')                                                                                                           #Pivotar a tabela

  g = tabela_freq.plot(kind='bar',                                              #Criar gráfico
                        color=['#808080']+['#800000'],                          #Escolher cores das barras
                        figsize=(25, 5),                                        #Escolher tamanho da figura
                        ylabel=False,                                           #Excluir rótulo do eixo y y
                        legend=None)                                            #Excluir legenda

  for rotulo in g.containers:                                                   #Criar rótulo de dados nas barras
        g.bar_label(rotulo,fontsize=15,padding=5)


  for spine in plt.gca().spines.values():                                       #Remover os eixos do gráfico
        spine.set_visible(False)
                          
  plt.xticks(rotation=0, fontsize=15)                                           #Excluir valores do eixo x
  plt.yticks([])                                                                #Aumentar a fonte do eixo y
  plt.xlabel(None)                                                              #Excluir rótulo do eixo y
  plt.ylabel(None)                                                              #Excluir rótulo do eixo y



  plt.title('Quantidade de pacientes que entraram ou não na UTI por faixa etária'+ 2*('\n'),fontsize=25,loc='left')           #Colocar o título
  plt.text(-0.5, 36,'A partir dos 70 anos a entrada na UTI é superior  ',fontsize=20,color='#800000')                         #Colocar o subtítulo


  plt.show()

### <font color= MediumSpringGreen> 2.5 grafico_genero_UTI
Esta função criará o gráfico comparativo entre os generos dos pacientes que tiveram ou não entrada na UTI.

In [ ]:
 def grafico_genero_UTI():
  tabela_freq = pd.DataFrame(data={'FAIXA_ETARIA': list(dados_sem_valores_unicos['GENDER']), 'UTI': list(dados_sem_valores_unicos['ICU'])})    #Criar dataframe
  tabela_freq['FAIXA_ETARIA'] = tabela_freq['FAIXA_ETARIA'].map({0:'Masculino',1:'Feminino'})                                            #Mapear valores dos rótulos
  tabela_freq = tabela_freq.value_counts().reset_index()                                                                                 #Criar frequência dos valores
  tabela_freq.columns = ['GENERO','UTI','QTD']                                                                                           #Renomear colunas
  tabela_freq = pd.pivot_table(tabela_freq,'QTD','GENERO','UTI')                                                                         #Pivotar a tabela


  g = tabela_freq.plot(kind='bar',                                              #Criar gráfico
                          color=['#808080']+['#800000'],                          #Escolher cores das barras
                          figsize=(15, 8),                                        #Escolher tamanho da figura
                          ylabel=False,                                           #Excluir rótulo do eixo y y
                          legend=None)                                            #Excluir legenda

  for rotulo in g.containers:                                                   #Criar rótulo de dados nas barras
          g.bar_label(rotulo,fontsize=15,padding=5)


  for spine in plt.gca().spines.values():                                       #Remover os eixos do gráfico
          spine.set_visible(False)
                            
  plt.xticks(rotation=0, fontsize=15)                                           #Excluir valores do eixo x
  plt.yticks([])                                                                #Aumentar a fonte do eixo y
  plt.xlabel(None)                                                              #Excluir rótulo do eixo y
  plt.ylabel(None)                                                              #Excluir rótulo do eixo y


  plt.title('Quantidade de pacientes que entraram ou não na UTI por gênero'+ 2*('\n'),fontsize=25,loc='left')           #Colocar o título
  plt.text(-0.5, 125,'O gênero masculino tem uma maior frequência de entrada na UTI',fontsize=20,color='#800000')                         #Colocar o subtítulo


  plt.show()

### <font color= MediumSpringGreen> 2.6 grafico_percentual_diferenca_sanguinea_5_menores
Esta função criará o gráfico comparativo entre as menores diferenças dos valores sanguíneos entre os pacientes que tiveram entrada na UTI sob o que não tiveram.

In [ ]:
def grafico_percentual_diferenca_sanguinea_5_menores():
  g = diferenca_percentual.tail(5).plot(kind='barh',                              #Escolher cores das barras
                                        color=['#808080'],
                                        figsize=(15, 8),                          #Escolher tamanho da figura
                                        ylabel=False,                             #Excluir rótulo do eixo y y
                                        legend=None)                              #Excluir legenda

  for rotulo in g.containers:                                                     #Criar rótulo de dados nas barras
        g.bar_label(rotulo,fontsize=15,padding=5)

  for spine in plt.gca().spines.values():                                         #Remover os eixos do gráfico
        spine.set_visible(False)
                          
  plt.xticks([])                                                                  #Excluir valores do eixo x
  plt.yticks(fontsize=15)                                                         #Aumentar a fonte do eixo y
  plt.xlabel(None)                                                                #Excluir rótulo do eixo y
  plt.ylabel(None)                                                                #Excluir rótulo do eixo y
  plt.xlim(-70,0)



  plt.text(-80,4.5,'Percentual de diferença dos indicadores sanguíneos dos pacientes'+ 2*('\n'),fontsize=25)           #Colocar o título
  plt.text(-80,4.2,'Top 5 menores entre os pacientes que tiveram entrada pelos que não tiveram  '+ 2*('\n'),fontsize=20)           #Colocar o título

  plt.show()

### <font color= MediumSpringGreen> 2.7 grafico_percentual_diferenca_sanguinea_5_maiores
Esta função criará o gráfico comparativo entre as maiores diferenças dos valores sanguíneos entre os pacientes que tiveram entrada na UTI sob o que não tiveram.

In [ ]:
def grafico_percentual_diferenca_sanguinea_5_maiores():
  diferenca_percentual.sort_values(by='mean',ascending=True,inplace=True)
  g = diferenca_percentual.tail(5).plot(kind='barh',                              #Escolher cores das barras
                                        color=['#808080'],
                                        figsize=(15, 8),                          #Escolher tamanho da figura
                                        ylabel=False,                             #Excluir rótulo do eixo y y
                                        legend=None)                              #Excluir legenda

  for rotulo in g.containers:                                                     #Criar rótulo de dados nas barras
        g.bar_label(rotulo,fontsize=15,padding=5)

  for spine in plt.gca().spines.values():                                         #Remover os eixos do gráfico
        spine.set_visible(False)
                          
  plt.xticks([])                                                                  #Excluir valores do eixo x
  plt.yticks(fontsize=15)                                                         #Aumentar a fonte do eixo y
  plt.xlabel(None)                                                                #Excluir rótulo do eixo y
  plt.ylabel(None)                                                                #Excluir rótulo do eixo y



  plt.text(-15,4.5,'Percentual de diferença dos indicadores sanguíneos dos pacientes'+ 2*('\n'),fontsize=25)           #Colocar o título
  plt.text(-15,4.2,'Top 5 maiores entre os pacientes que tiveram entrada pelos que não tiveram  '+ 2*('\n'),fontsize=20)           #Colocar o título

  plt.show()

## <font color= MediumSpringGreen> 3. Funções de treino e desenvolvimento do modelo



### <font color= MediumSpringGreen> 3.1 roda_modelo
Esta função rodará o modelo, imprimirá o AUC médio e o relatório da classificação.

In [ ]:
def roda_modelo(modelo,dados):

  np.random.seed(354354)

  x_columns = dados.columns
  y = dados['ICU']
  x = dados[x_columns].drop(['ICU','WINDOW'],axis=1)
  
  x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y, test_size=0.15)

  modelo.fit(x_train,y_train)
  predicao = modelo.predict(x_test)
  prob_predict = modelo.predict_proba(x_test)

  auc = roc_auc_score(y_test,prob_predict[:,1])
  print(f"AUC {auc}")
  print("\nClassification Report")
  print(classification_report(y_test,predicao))

### <font color= MediumSpringGreen> 3.2 rodroda_n_modelos
Esta função rodará n vezes o modelo e imprimirá o intervalo do auc médio gerado.

In [ ]:
def roda_n_modelos(modelo,dados,n):

  np.random.seed(354354)

  x_columns = dados.columns
  y = dados['ICU']
  x = dados[x_columns].drop(['ICU','WINDOW'],axis=1)

  auc_lista = []
  for _ in range(n):

    x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y,test_size=0.15)

    modelo.fit(x_train,y_train)
    prob_predict = modelo.predict_proba(x_test)
    auc = roc_auc_score(y_test, prob_predict[:,1])
    auc_lista.append(auc)
  
  
  auc_medio = np.mean(auc_lista)
  auc_std = np.std(auc_lista)
  print(f'AUC médio {auc_medio}')
  print(f'Intervalo {auc_medio - 2* auc_std} - {auc_medio + 2* auc_std}')

### <font color= MediumSpringGreen> 3.3 roda_modelo_cv
Esta função rodará o modelo através de uma validação cruzada utilizando o StratifiedKFold.

In [ ]:
def roda_modelo_cv(modelo,dados,n_splits,n_repeat):

  np.random.seed(354354)
  dados = dados.sample(frac=1).reset_index(drop=True)
  x_columns = dados.columns
  y = dados['ICU']
  x = dados[x_columns].drop(['ICU','WINDOW'],axis=1)
  
  cv = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats=n_repeat)
  resultados = cross_validate(modelo,x,y,cv=cv, scoring='roc_auc',return_train_score=True)

  auc_medio = np.mean(resultados['test_score'])
  auc_medio_treino = np.mean(resultados['train_score'])

  auc_std = np.std(resultados['test_score'])


  #print(f"AUC Médio {auc_medio} Intervalo {auc_medio - (2*auc_std)} - {auc_medio + (2*auc_std)}")
  print(f"AUC {auc_medio} - {auc_medio_treino}")
  return auc_medio, auc_medio_treino